### 1 DEMO

In [2]:
import torch
from gpt_model import GPT

if __name__ == '__main__':
    chooseEpoch = 28 # 
    modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'

    device = torch.device('cuda')
    model = GPT().to(device)
    model.load_state_dict(torch.load(modelPath))
    model.eval()
    
    #初始输入是空，每次加上后面的对话信息
    sentence = ''
    while True:
        temp_sentence = input("我:")
        sentence += (temp_sentence + '\t')
        if len(sentence) > 200:
            #由于该模型输入最大长度为300，避免长度超出限制长度过长需要进行裁剪
            t_index = sentence.find('\t')
            sentence = sentence[t_index + 1:]
        print("我:", temp_sentence)
        print("机器人:", model.answer(sentence))


我: 亲爱的几时出去
机器人: 我也不知道啊，我们都是去看电影的


KeyboardInterrupt: Interrupted by user

## 2 Show Model Answers in QAData

In [3]:
import json
import torch
import time
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
from nltk.translate.bleu_score import corpus_bleu
import torch
from gpt_model import GPT

def getQuesGoldAns(validQAPath):
    Questions = []
    GoldenAnswers = []
    with open(validQAPath, 'r', encoding='utf-8') as src_file:
        for line in src_file:
            QA = line.strip().split('\t')
            Questions.append(QA[0])
            GoldenAnswers.append(QA[1])

    return Questions,GoldenAnswers


if __name__ == '__main__':
    evaEpochList = [28]
    for e in evaEpochList:
        chooseEpoch = e # 
        modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'
        validQAPath = 'dataset/QA_valid_100.txt'

        device = torch.device('cuda')
        model = GPT().to(device)
        model.load_state_dict(torch.load(modelPath))
        model.eval()

        # evaluation
        print("Epoch %d Ans show:"%(chooseEpoch), end='')
        Questions,GoldenAnswers = getQuesGoldAns(validQAPath)
        ModelAnswers = []
        for Q in tqdm(Questions):
            A = model.answer(Q+'\t').strip()
            ModelAnswers.append(A)
        
        for q,a in zip(Questions, ModelAnswers):
            print("Ask: %s\nModelAns: %s"%(q,a))

Epoch 12 Ans show:

100%|██████████| 100/100 [00:19<00:00,  5.03it/s]

Ask: 啊我好爱虾仁蛋黄酱金枪鱼蛋黄酱
ModelAns: 我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我也是，我
Ask: 考试撞墙关驾校屁事？你怎么不顺便把考场施工单位也告了？
ModelAns: 你这是在黑河南的节奏吗？
Ask: 看了下全文，那女的考试当天就表明身体不舒服了，考试不是她预约是教练自己安排的，教练还让她考试不就是教练的错吗？而且她住院花了31万，赔30万不过分吧
ModelAns: 我们这边的考试不是她的错，她是她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的错，她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的惩罚她的
Ask: 我先去穿衣服，准备走了
ModelAns: 我也想去
Ask: 好好好，偶遇我大闺蜜
ModelAns: 我也是，我们一起去的
Ask: 期待小猎豹的表现
ModelAns: 我也是
Ask: 为了你们新年有惊喜，我也是用心良苦了
ModelAns: 新年快乐
Ask: 世界上最远的距离，不是爱，不是恨，而是熟悉的人，渐渐变得陌生。
ModelAns: 是的，我们都在，我们在的是不是爱，但是我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我们在的是不是爱，我
Ask: 生日快乐！年年18
ModelAns: 谢谢
Ask: 你是18那我17好了
ModelAns: 我17
Ask: 青青妹！！！好久不见！别来无恙！！假期也没见你呜呜呜！我不想你，谁想你谁小狗呜呜呜呜呜呜呜！汪！
ModelAns: 我也是！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
Ask: 文艺小青年
ModelAns: 你是不是傻
Ask: 你知道这期录的什么时候播不
ModelAns: 不知道
Ask: 不知道
ModelAns: 我也是
Ask: 好吧，我去建房子了
ModelAns: 你去

## 3 EVALUATION SAMPLE FOR BERT SCORE

In [7]:
# sentence-lv bert_score sample
from bert_score import score as bert_score

ModelAnswers = ['我们都是星期天交的', '我们都是星期一交的']
GoldenAnswers = ['我们都是星期天交的', '我们都是星期天交的']

P, R, F1 = bert_score(ModelAnswers, GoldenAnswers, lang='zh')
avg_bert_score = F1.mean().item()
print(avg_bert_score)

0.9871877431869507


## 4 Get Model Answers for test set and BERT SCORE Evaluation

In [2]:
import json
import torch
import time
from tqdm import tqdm
from bert_score import score as bert_score
from gpt_model import GPT

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def getQuesGoldAns(validQAPath):
    Questions = []
    GoldenAnswers = []
    with open(validQAPath, 'r', encoding='utf-8') as src_file:
        for line in src_file:
            QA = line.strip().split('\t')
            Questions.append(QA[0])
            GoldenAnswers.append(QA[1])

    return Questions,GoldenAnswers



if __name__ == '__main__':
    evaEpochList = list(range(28, 29))
    
    best_bert_score = -1
    prev_bert_score = -1
    no_improvement_count = 0
    all_bert_score_evaluation_info = []

    for e in evaEpochList:
        chooseEpoch = e # 
        modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'
        validQAPath = 'dataset/QA_test.txt'

        device = torch.device('cuda')
        model = GPT().to(device)
        model.load_state_dict(torch.load(modelPath))
        model.eval()

        # evaluation
        print("Epoch %d Evaluation:"%(chooseEpoch), end='')
        start_time = time.time()
        Questions,GoldenAnswers = getQuesGoldAns(validQAPath) 
        ModelAnswers = []
        for Q in tqdm(Questions):
            A = model.answer(Q+'\t').strip()
            ModelAnswers.append(A)
        
        with open('bestModelAns4Test.json', 'w') as json_file:
            json.dump(ModelAnswers, json_file)
        # print(GoldenAnswers[:5])
        # print(ModelAnswers[:5])

        # 接下来要使这个东西连续三次不上升就跳出循环
        P, R, F1 = bert_score(ModelAnswers, GoldenAnswers, lang='zh')
        avg_bert_score = F1.mean().item()
        end_time = time.time()
        eva_mins, eva_secs = epoch_time(start_time, end_time)

        if best_bert_score <= avg_bert_score:
            best_bert_score = avg_bert_score

        if prev_bert_score > avg_bert_score:
            no_improvement_count += 1
        else:
            no_improvement_count = 0 

        bert_score_evaluation_info = {
            'epoch': chooseEpoch,
            'bert_score': avg_bert_score,
            'best_bert_score': best_bert_score,
            'no_improvement_count': no_improvement_count
        }

        all_bert_score_evaluation_info.append(bert_score_evaluation_info)

        with open('models/bert_score_evaluation_info.json', 'w') as json_file:
            json.dump(all_bert_score_evaluation_info, json_file)

        prev_bert_score = avg_bert_score

        print(f' | Time: {eva_mins}m {eva_secs}s')
        print("avg_bert_score= %f | best_bert_score= %f | no_improvement_count = %d"%(avg_bert_score,best_bert_score,no_improvement_count))



Epoch 28 Evaluation:

100%|██████████| 19008/19008 [12:25<00:00, 25.48it/s]


 | Time: 13m 8s
avg_bert_score= 0.572730 | best_bert_score= 0.572730 | no_improvement_count = 0


## 5 EVALUATION FOR BLEU

In [4]:
import json
import torch
import time
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
from nltk.translate.bleu_score import corpus_bleu

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def getQuesGoldAns(validQAPath):
    Questions = []
    GoldenAnswers = []
    with open(validQAPath, 'r', encoding='utf-8') as src_file:
        for line in src_file:
            QA = line.strip().split('\t')
            Questions.append(QA[0])

            ga = QA[1]
            ga = list(ga)
            ga = " ".join(ga)
            ga = [wordpunct_tokenize(ga)]
            GoldenAnswers.append(ga)

    return Questions,GoldenAnswers

def GPT_BLUE_Evaluation(GPTModelPath, QAPath):
    device = torch.device('cuda')
    model = GPT().to(device)
    model.load_state_dict(torch.load(GPTModelPath))
    model.eval()

    Questions,GoldenAnswers = getQuesGoldAns(QAPath)
    ModelAnswers = []
    for Q in tqdm(Questions):
        A = model.answer(Q+'\t').strip()
        A = list(A)
        A = " ".join(A)
        A = wordpunct_tokenize(A)
        ModelAnswers.append(A)
    
    # print(GoldenAnswers[:5])
    # print(ModelAnswers[:5])

    bleu_score = corpus_bleu(GoldenAnswers, ModelAnswers) # 接下来要使这个东西连续三次不上升就跳出循环

    return bleu_score

if __name__ == '__main__':
    evaEpochList = [28]
    for e in evaEpochList:
        chooseEpoch = e # 
        modelPath = 'models/GPT2_epoch'+str(chooseEpoch)+'.pt'
        validQAPath = 'dataset/QA_test.txt'

        device = torch.device('cuda')
        model = GPT().to(device)
        model.load_state_dict(torch.load(modelPath))
        model.eval()

        # evaluation
        print("Epoch %d Evaluation:"%(chooseEpoch), end='')
        start_time = time.time()
        Questions,GoldenAnswers = getQuesGoldAns(validQAPath)
        ModelAnswers = []
        with open('bestModelAns4Test.json', 'r') as json_file:
            befModelAnswers = json.load(json_file)
        for Q in tqdm(befModelAnswers):
            A = Q.strip()
            A = list(A)
            A = " ".join(A)
            A = wordpunct_tokenize(A)
            ModelAnswers.append(A)
        
        print(GoldenAnswers[:5])
        print(ModelAnswers[:5])

        bleu_score = corpus_bleu(GoldenAnswers, ModelAnswers) # 接下来要使这个东西连续三次不上升就跳出循环
        end_time = time.time()
        eva_mins, eva_secs = epoch_time(start_time, end_time)
        print(f' | Time: {eva_mins}m {eva_secs}s')
        print("\tBLEUScore= %f"%(bleu_score))

Epoch 28 Evaluation:

100%|██████████| 19008/19008 [00:00<00:00, 237130.27it/s]

[[['去', '相', '机', '家', '里', '吃', '…', '…']], [['相', '机', '今', '年', '木', '有', '回', '去', 'T', '.', 'T']], [['你', '过', '来', '我', '们', '什', '么', '关', '系']], [['我', '不', '挑', '食']], [['死', '鱼', '皮', '真', '会', '安', '慰', '人', '那', '不', '是', '翘', '臀', '是', '肥', '肉', '不', '！', '是', '赘', '肉', '！']]]
[['我', '也', '饿', '了'], ['我', '也', '想', '吃'], ['我', '想', '要', '的', '是', '爱', '情', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是', '永', '远', '不', '会', '变', '的', '，', '爱', '情', '是'], ['老', '铁', '家', '好', '吃'], ['我', '的', '脸', '是', '小', '了', '，', '你', '的', '脸', '是', '不', '是', '很', '小', '了']]


 | Time: 0m 0s
	BLEUScore= 0.008305


## combine training_info jsons

In [1]:
import json
# define the first and second json data
json1 = [
    {
        "epoch": 1,
        "train_time": "54m 20s",
        "train_loss": 3.660306632578193
    }
]

json2 = [
    {
        "epoch": 1,
        "bert_score": 0.5535060167312622,
        "best_bert_score": 0.5535060167312622,
        "no_improvement_count": 0
    }
]

with open('models/training_info.json', 'r', encoding='utf-8') as f:
    json1 = json.load(f)
with open('models/bert_score_evaluation_info.json', 'r', encoding='utf-8') as f:
    json2 = json.load(f)

# convert json data to dictionary where key is 'epoch'
dict1 = {j['epoch']: j for j in json1}
dict2 = {j['epoch']: j for j in json2}

# merge dictionaries
merged = {epoch: {**dict1[epoch], **dict2[epoch]} for epoch in dict1}

# convert back to json
merged_json = list(merged.values())

print(merged_json)

with open('Results/results.json', 'w', encoding='utf-8') as f:
    json.dump(merged_json, f)

[{'epoch': 1, 'train_time': '54m 20s', 'train_loss': 3.660306632578193, 'bert_score': 0.5535060167312622, 'best_bert_score': 0.5535060167312622, 'no_improvement_count': 0}, {'epoch': 2, 'train_time': '53m 36s', 'train_loss': 3.3505166824188066, 'bert_score': 0.5551387071609497, 'best_bert_score': 0.5551387071609497, 'no_improvement_count': 0}, {'epoch': 3, 'train_time': '53m 20s', 'train_loss': 3.258897390724891, 'bert_score': 0.5606464147567749, 'best_bert_score': 0.5606464147567749, 'no_improvement_count': 0}, {'epoch': 4, 'train_time': '53m 19s', 'train_loss': 3.2071923815236634, 'bert_score': 0.5622338056564331, 'best_bert_score': 0.5622338056564331, 'no_improvement_count': 0}, {'epoch': 5, 'train_time': '53m 20s', 'train_loss': 3.172341417046756, 'bert_score': 0.5625004768371582, 'best_bert_score': 0.5625004768371582, 'no_improvement_count': 0}, {'epoch': 6, 'train_time': '55m 38s', 'train_loss': 3.1463054955975323, 'bert_score': 0.5632643103599548, 'best_bert_score': 0.5632643103